### 크롤링한 악성코드 CSV에 있는 API 목록들을 예쁘게 TXT로 저장

In [ ]:
import csv

api_list = []

with open("test.csv", "r") as f:
    reader = csv.reader(f)

    for text in reader:
        if len(text) == 0:
            continue
        if text[0] == "API":
            continue
        api_list.append(text[0])
    f.close()

with open("test.txt", "w") as f:
    for txt in api_list:
        f.write(txt + '\n')
    f.close()

### 분석한 악성코드별 API 중복제거 및 원본 리스트 저장

In [ ]:
# 중복제거된 API
api1_overlap = set()
api2_overlap = set()
api3_overlap = set()

# 원본 API 리스트
api1_origin = []
api2_origin = []
api3_origin = []

In [ ]:
with open("test1.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        api1_overlap.add(line)
        api1_origin.append(line)
        
with open("test2.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        api2_overlap.add(line)
        api2_origin.append(line)
        
with open("test3.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        api3_overlap.add(line)
        api3_origin.append(line)        

In [ ]:
len(api1_overlap), len(api2_overlap), len(api3_overlap)

In [ ]:
len(api1_origin), len(api2_origin), len(api3_origin)

### 세개 악성코드에서 중복되는 API 추출

In [ ]:
api_intersection = api1_overlap & api2_overlap & api3_overlap
len(api_intersection)

### 화이트 리스트 API 추출하기

In [ ]:
api_white = set()
with open("white.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        for i in line.split(", "):
            api_white.add(i)
            
len(api_white)

### 악성코드 API 교집합에서 화이트 리스트 API 제거하기

In [ ]:
api_malware = api_intersection - api_white
len(api_malware)

### 후킹이 어려운 NTdll API 제거하기

In [ ]:
api_final = [w for w in api_malware if w[:2]!="Nt"]
len(api_final)

### 악성코드 별, 교집합에 해당하는 API 순서 패턴 추출하기

In [ ]:
api1_inter_order = []
api2_inter_order = []
api3_inter_order = []

In [ ]:
# 교집합에 속하는 API들을 순서대로 추출
for api in api1_origin:
    if api in api_final:
        api1_inter_order.append(api)
        
for api in api2_origin:
    if api in api_final:
        api2_inter_order.append(api)
        
for api in api3_origin:
    if api in api_final:
        api3_inter_order.append(api)

In [ ]:
len(api1_inter_order), len(api2_inter_order), len(api3_inter_order) 

In [ ]:
# 순서를 유지한 채로 중복 제거
api1_order = list(dict.fromkeys(api1_inter_order))
api2_order = list(dict.fromkeys(api2_inter_order))
api3_order = list(dict.fromkeys(api3_inter_order))

In [ ]:
len(api1_order), len(api2_order), len(api3_order) 

### api numbering

In [ ]:
api_numbering = {key:i for i, key in enumerate(api_final)}
api_reversing = {i:key for i, key in enumerate(api_final)}

### 악성코드 api 핵심 순서 도출

In [ ]:
print("<api1>")
for z in api1_order:
    print(api_numbering[z], end=" => ")
print(end="\n\n")    

print("<api2>")
for z in api2_order:
    print(api_numbering[z], end=" => ")
print(end="\n\n")        
    
print("<api3>")
for z in api3_order:
    print(api_numbering[z], end=" => ")    
print(end="\n\n")        

## 시나리오 패턴 도식화

해당 논문을 참고하여, api1, api2, api3에 공통적으로 해당하는 api 패턴을 추출하여, 시나리오로 결정하였습니다.

논문 링크 : http://index-of.co.uk/Reverse-Engineering/API.pdf

두가지 방법으로 패턴 추출을 진행하였고, 그 결과 다음과 같은 결론을 도출하였습니다.

#### api1

31 => 32 => 13 => 24 => 2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 => 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

#### api2

24 => 32 => 2 => 3 => 7 => 20 => 13 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 => 31 => 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

#### api3

31 => 32 => 13 => 24 => 2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 => 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

### 방법 1: 최대한 가장 긴 api 패턴을 추출하자

#### api 패턴1 

32 => 2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 =>10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

api 패턴1이 등장할 경우, 해당 프로세스를 **91% (31/34)의 확률**로 키로그 악성코드로 판단하기로 하였다.

### 방법 2: 제일 먼저 등장하는 api 패턴을 추출하자.

#### api 패턴1

31 = > 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

#### api 패턴2

2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29

#### api 패턴3

32 => 13

api 패턴1이 등장할 경우, 해당 프로세스를 **50% (17/34)의 확률**로 키로그 악성코드로 판단하기로 하였다.

api 패턴2이 등장할 경우, 해당 프로세스를 **41% (14/34)의 확률**로 키로그 악성코드로 판단하기로 하였다.

api 패턴3이 등장할 경우, 해당 프로세스를 **5.8% (2/34)의 확률**로 키로그 악성코드로 판단하기로 하였다.

### 결론

확률이 높은 3가지 패턴을 키로그 악성코드 시나리오로 채택하게 되었다.

#### 시나리오1
32 => 2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 =>10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

#### 시나리오2
31 = > 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14

#### 시나리오3
2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29

### 숫자를 문자로 매칭

In [ ]:
scenario1 = "32 => 2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29 => 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14"
scenario2 = "31 => 10 => 22 => 21 => 18 => 6 => 27 => 28 => 4 => 9 => 17 => 0 => 12 => 5 => 1 => 16 => 14"
scenario3 = "2 => 3 => 7 => 20 => 25 => 8 => 11 => 33 => 30 => 19 => 15 => 26 => 23 => 29"

In [ ]:
print("scenario1", end="\n\n")
for scenario in scenario1.split(" => "):
    print(api_reversing[int(scenario)].strip())
print()
    
print("scenario2", end="\n\n")
for scenario in scenario2.split(" => "):
    print(api_reversing[int(scenario)].strip())
print()

print("scenario3", end="\n\n")
for scenario in scenario3.split(" => "):
    print(api_reversing[int(scenario)].strip())    
print()    

## 최종결론

우리는 위와 같은 절차를 통해 3가지의 키로깅 악성코드의 행위기반 시나리오를 작성할 수 있었다.

#### 시나리오1 (91% 확률로 키로거라고 판단)
1. LdrGetDllHandle
2. LdrGetProcedureAddress
3. RegOpenKeyExW
4. RegQueryInfoKeyW
5. RegEnumKeyExW
6. RegEnumValueW
7. RegCloseKey
8. RegQueryValueExW
9. FindFirstFileExW
10. memcpy
11. VirtualProtectEx
12. GetSystemInfo
13. GlobalMemoryStatus
14. GetFileVersionInfoSizeW
15. GetFileVersionInfoW
16. HeapCreate
17. IsDebuggerPresent
18. CreateThread
19. RtlAddVectoredExceptionHandler
20. SetUnhandledExceptionFilter
21. GlobalMemoryStatusEx
22. LsaOpenPolicy
23. UrlCanonicalizeW
24. StrCmpNICW
25. DeviceIoControl
26. RegOpenKeyExA
27. SHGetFolderPathW
28. RegQueryValueExA
29. CryptAcquireContextW
30. CryptGenRandom
31. RegCreateKeyExW

#### 시나리오2 (50% 확률로 키로거라고 판단)
1. GetSystemTimeAsFileTime
2. HeapCreate
3. IsDebuggerPresent
4. CreateThread
5. RtlAddVectoredExceptionHandler
6. SetUnhandledExceptionFilter
7. GlobalMemoryStatusEx
8. LsaOpenPolicy
9. UrlCanonicalizeW
10. StrCmpNICW
11. DeviceIoControl
12. RegOpenKeyExA
13. SHGetFolderPathW
14. RegQueryValueExA
15. CryptAcquireContextW
16. CryptGenRandom
17. RegCreateKeyExW

#### 시나리오3 (41% 확률로 키로거라고 판단)
1. LdrGetProcedureAddress
2. RegOpenKeyExW
3. RegQueryInfoKeyW
4. RegEnumKeyExW
5. RegEnumValueW
6. RegCloseKey
7. RegQueryValueExW
8. FindFirstFileExW
9. memcpy
10. VirtualProtectEx
11. GetSystemInfo
12. GlobalMemoryStatus
13. GetFileVersionInfoSizeW
14. GetFileVersionInfoW

이러한 시나리오를 기반으로 특정한 API를 후킹하여, 키로거 악성코드를 탐지하는 Dectector 프로그램을 제작하도록 하겠다.